In [1]:
import findspark
findspark.init()

In [2]:
import datetime
import pickle
from urllib.request import urlopen
import os
import pandas as pd
import numpy as np
import gensim
import pickle
import matplotlib.pyplot as plt

/Users/User-MC/opt/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/Galeforse/DST-Assessment-05/main/Data/NCSC%20Reports.csv')

In [4]:
df.head()

,Unnamed: 0,Title,Article,topics,Links
0,0,23rd April 2021,['The NCSC is aware that a malicious piece of ...,"['Cyber attack', 'Cyber strategy', 'Education'...",https://www.ncsc.gov.uk/report/weekly-threat-r...
1,1,16th April 2021,['Cyber security researchers have uncovered a ...,"['Cyber strategy', 'Patching', 'Vulnerabilities']",https://www.ncsc.gov.uk/report/weekly-threat-r...
2,2,12th April 2021,"['Cyber security researchers, Esentire, have w...","['Phishing', 'Social media', 'Personal data', ...",https://www.ncsc.gov.uk/report/weekly-threat-r...
3,3,2nd April 2021,['The UK education sector continues to face an...,"['Education', 'Incident management', 'Secure d...",https://www.ncsc.gov.uk/report/weekly-threat-r...
4,4,26th March 2021,['Earlier this month Microsoft confirmed that ...,"['Cyber attack', 'Education', 'Mitigation', 'P...",https://www.ncsc.gov.uk/report/weekly-threat-r...


In [5]:
print(np.shape(df))

(221, 5)


In [6]:
df.iloc[4,2]

"['Earlier this month Microsoft confirmed that sophisticated actors had attacked a number of Exchange servers and have released multiple security updates since.\\nThe NCSC is strongly urging any organisation using affected versions to proactively search their system for evidence of compromise, in line with Microsoft guidance.\\nOur alert was updated last week with the most recent advice and guidance.', 'A new report has revealed that almost 40% of UK businesses experienced a cyber attack last year.', 'The Cyber Security Breaches Survey 2021 report, published by the Department of Digital, Culture, Media and Sport (DCMS), also revealed that a quarter of charities have suffered a cyber security breach or attack in the past 12 months.', 'The NCSC has produced guidance to address some of the most common breaches and attacks identified in the report:\\nphishing emails\\nviruses\\nransomware\\nWhere a breach has resulted in a loss of data or assets, the average cost of a cyber attack on a bus

In [7]:
import nltk
from nltk.corpus import stopwords

In [8]:
l1 = stopwords.words('english')
l2 = ['have', 'with', 'cannot', 'all', 'more', 'less', 'fewer', 'if', 'from', 'new', 'old', 'be', 'This']
l1.append(l2)
lst1 = [' {} '.format(x) for x in l1]

len(lst1)

180

In [9]:
df['Article'].replace(lst1, " ", regex=True, inplace=True)
df['Article'].replace(r'[^\w\s]', " ", regex=True, inplace=True)
df['Article'].replace('[0-9999999999]', " ", regex=True, inplace=True)

In [10]:
df.iloc[4,2]

'  Earlier month Microsoft confirmed sophisticated actors attacked number Exchange servers released multiple security updates since  nThe NCSC strongly urging organisation using affected versions proactively search system evidence compromise  line Microsoft guidance  nOur alert updated last week recent advice guidance     A new report revealed almost     UK businesses experienced cyber attack last year     The Cyber Security Breaches Survey      report  published Department Digital  Culture  Media Sport  DCMS   also revealed quarter charities suffered cyber security breach attack past    months     The NCSC produced guidance address common breaches attacks identified report  nphishing emails nviruses nransomware nWhere breach resulted loss data assets  average cost cyber attack business         This figure rises         medium large businesses  nThe report also shows cyber risk organisations heightened pandemic  made securing digital environments challenging resources diverted facilita

In [11]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [12]:
nltk.download('wordnet')
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

[nltk_data] Downloading package wordnet to /Users/User-MC/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [13]:
df_lem = df['Article'].apply(lemmatize_text)

In [14]:
df_lem

0      [The, NCSC, aware, malicious, piece, spyware, ...
1      [Cyber, security, researcher, uncovered, serie...
2      [Cyber, security, researcher, Esentire, warned...
3      [The, UK, education, sector, continues, face, ...
4      [Earlier, month, Microsoft, confirmed, sophist...
                             ...                        
216    [Malware, infected, ATMs, compromise, Indian, ...
217    [Threat, assessment, trend, analysis, nOnline,...
218    [New, Trojan, used, financial, attack, nSymant...
219    [Threat, assessment, trend, analysis, nDressco...
220    [Threat, assessment, trend, analysis, nYahoo, ...
Name: Article, Length: 221, dtype: object

In [15]:
df_lem_list = df_lem.values.tolist()

In [16]:
lister = [[] for x in range(len(df_lem))]
for i in range(len(df_lem_list)):
    l = df_lem_list[i]
    for token in l:
        if (len(token)>4 or token.isupper()):
            lister[i].append(token)

lister[0:1]

[['NCSC',
  'aware',
  'malicious',
  'piece',
  'spyware',
  'known',
  'FluBot',
  'affecting',
  'Android',
  'phone',
  'device',
  'across',
  'UK',
  'spyware',
  'installed',
  'victim',
  'receives',
  'message',
  'asking',
  'install',
  'tracking',
  'missed',
  'package',
  'delivery',
  'nScammers',
  'cyber',
  'criminal',
  'regularly',
  'exploit',
  'known',
  'trusted',
  'brand',
  'personal',
  'FluBot',
  'campaign',
  'prime',
  'example',
  'nAndroid',
  'urged',
  'familiarise',
  'guidance',
  'vigilant',
  'suspicious',
  'looking',
  'message',
  'forwarded',
  'Earlier',
  'FireEye',
  'published',
  'saying',
  'APT',
  'actor',
  'actively',
  'exploiting',
  'vulnerability',
  'Pulse',
  'Connect',
  'VPN',
  'appliance',
  'follows',
  'security',
  'update',
  'Pulse',
  'Secure',
  'recently',
  'discovered',
  'limited',
  'number',
  'customer',
  'experienced',
  'evidence',
  'exploit',
  'behaviour',
  'Pulse',
  'Connect',
  'Secure',
  'PCS',
  

In [17]:
dictionary = gensim.corpora.Dictionary(lister)

In [18]:
bow_corpus = [dictionary.doc2bow(doc) for doc in lister]
tfidf=gensim.models.TfidfModel(bow_corpus)
tfidf_corp = tfidf[bow_corpus]

In [19]:
bow_corpus[0:1]

[[(0, 1),
  (1, 1),
  (2, 1),
  (3, 1),
  (4, 1),
  (5, 1),
  (6, 3),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 1),
  (14, 2),
  (15, 1),
  (16, 3),
  (17, 1),
  (18, 1),
  (19, 4),
  (20, 3),
  (21, 1),
  (22, 4),
  (23, 1),
  (24, 3),
  (25, 2),
  (26, 2),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 2),
  (34, 1),
  (35, 1),
  (36, 3),
  (37, 1),
  (38, 2),
  (39, 2),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 3),
  (47, 1),
  (48, 5),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 2),
  (64, 1),
  (65, 2),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 2),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 2),
  (79, 1),
  (80, 1),
  (81, 1),
  (82, 1),
  (83, 2),
  (84, 1),
  (85, 1),
  (86, 1),
  (87, 1),
  (88, 1),
  (89, 1),
  (90, 1),
  (91, 2)

In [20]:
lda_model_tfidf = gensim.models.LdaMulticore(tfidf_corp, num_topics=6, id2word=dictionary, passes=2, workers=4)

In [27]:
!pip install pyldavis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

ModuleNotFoundError: No module named 'pyLDAvis.gensim'

In [ ]:
lda_display = pyLDAvis.gensim.prepare(lda_model_tfidf, tfidf_corp, 
                                          dictionary, mds='mmds')
pyLDAvis.display(lda_display, template_type='notebook')

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
def getCoherence(m,t,d):
    coherence_model_lda = CoherenceModel(model=m,texts=t, dictionary=d, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return(coherence_lda) # Need high coherence

In [ ]:
getCoherence(m=lda_model_tfidf, t=lister, d =dictionary)

In [ ]:
perplex = lda_model_tfidf.log_perplexity(tfidf_corp)

In [ ]:
perplex

In [ ]:
begin_time = datetime.datetime.now()
gCH = [0] * 50
for i in range(1,50):
    locals()['lda_model_tfidf_{}'.format(i)] = gensim.models.LdaMulticore(tfidf_corp, num_topics=i, id2word=dictionary, passes=2, workers=1)
    gCH[i]=getCoherence(m=locals()['lda_model_tfidf_{}'.format(i)], t=lister, d =dictionary)
end_time = datetime.datetime.now()
time_weak_coh = (end_time - begin_time)

print(time_weak_coh)

In [ ]:
begin_time = datetime.datetime.now()
gCH = [0] * 50
for i in range(1,50):
    locals()['lda_model_tfidf_{}'.format(i)] = gensim.models.LdaMulticore(tfidf_corp, num_topics=i, id2word=dictionary, passes=2, workers=6)
    gCH[i]=getCoherence(m=locals()['lda_model_tfidf_{}'.format(i)], t=lister, d =dictionary)
end_time = datetime.datetime.now()
time_coh = (end_time - begin_time)

print(time_coh)

In [ ]:
begin_time = datetime.datetime.now()
perplexity = [0] * 50
for i in range(1,50):
    perplexity[i]=locals()['lda_model_tfidf_{}'.format(i)].log_perplexity(tfidf_corp)
end_time = datetime.datetime.now()
time_perp = (end_time - begin_time)

print(time_perp)

In [ ]:
plt.plot(gCH)
plt.show()
plt.plot(perplexity)
plt.show()

In [ ]:
print("The maximum coherence achieved is", max(gCH), "and its corresponding number of topics is", np.argmax(gCH))
print("The minimum perplexity achieved is", min(perplexity), "and its corresponding number of topics is", np.argmin(perplexity))

In [ ]:
lda_display = pyLDAvis.gensim.prepare(lda_model_tfidf_16, tfidf_corp, 
                                          dictionary, mds='mmds')
pyLDAvis.display(lda_display, template_type='notebook')

## pyspark

In [ ]:
# !pip create -n sparknlp python=3.7 -y
# !pip activate sparknlp
# !pip install spark-nlp==3.0.2 pyspark==3.1.1

In [ ]:
import sparknlp
sparknlp = sparknlp.start()

In [ ]:
# from pyspark import SparkContext
# print(SparkContext.version)

# sc = SparkContext()
# sc

In [ ]:
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,
                                Lemmatizer, StopWordsCleaner)
from pyspark.ml import Pipeline

In [ ]:
eng_stopwords = stopwords.words('english')

In [ ]:
documentAssembler = DocumentAssembler() \
     .setInputCol('Article') \
     .setOutputCol('Articles')

tokenizer = Tokenizer() \
     .setInputCols(['Articles']) \
     .setOutputCol('token')

normalizer = Normalizer() \
     .setInputCols(['token']) \
     .setOutputCol('normalized') \
     .setLowercase(True)

lemmatizer = Lemmatizer() \
     .setInputCols(['normalized']) \
     .setOutputCol('lemma') \
     .setDictionary('https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt', key_delimiter = '->', value_delimiter = '\t')

stopwords_cleaner = StopWordsCleaner() \
     .setInputCols(['lemma']) \
     .setOutputCol('clean_lemma') \
     .setCaseSensitive(False) \
     .setStopWords(eng_stopwords)

finisher = Finisher() \
     .setInputCols(['clean_lemma']) \
     .setCleanAnnotations(False)

In [ ]:
pipeline = Pipeline() \
     .setStages([
           documentAssembler,
           tokenizer,
           normalizer,
           lemmatizer,
           stopwords_cleaner,
           finisher
     ])

In [ ]:
df.to_csv('NCSC Reports.csv')

In [ ]:
#df = sparknlp.read.format("NCSC Reports.csv", format="csv", header="true")
# df = sparknlp.sql("SELECT * FROM csv.`NCSC Reports.csv`")
dfs = sparknlp.read.load("NCSC Reports.csv",format='csv', header = "true")

In [ ]:
dfs.printSchema()

In [ ]:
pd.DataFrame(dfs.take(10), columns=dfs.columns)

In [ ]:
NCSC_pipeline = pipeline.fit(dfs).transform(dfs)